In [25]:
import numpy as np  
from asteroloc8.asteroestimate import detections
from asteroloc8.asteroestimate.detections import probability as prob                                               
from scipy.stats import norm, multivariate_normal


class NuPrior(object):
    '''                                                                                              
    Provide guesses for numax using three different methods and also optionally numax prior distributions.
    1) specnmx()
     Uses spectroscopic log g + spectroscopic temperature.
    2) gaiascalnmx()
     Uses Gaia parallax + apparent magnitude + bolometric correction + photometric temperature + optional extinction.
    3) gaiamlnmx():
     Uses a data-driven approach to map Gaia luminosity to numax.
    '''
    
    def __init__(self, plx=None, plx_err=None, logg_spec=None, logg_spec_err=None, teff_spec=None, teff_spec_err=None,
                 jmag=None, jmag_err=None, hmag=None, hmag_err=None, kmag=None, kmag_err=None):
        ''' 
        INPUTS:                                                                                              
        [ plx, plx_err : float, float ]
         Parallax and uncertainty [mas]. Default None.
        [ logg_spec, logg_spec_err : float, float ]
         Spectroscopic log g and uncertainty [cgs]. Default None.
        [ teff_spec, teff_spec_err : float, float ]
         Spectroscopic temperature and uncertainty [K]. Default None.  
        [ jmag, jmag_err : float, float ]
         J-band magnitude and uncertainty [mag]. Default None.
        [ hmag, hmag_err : float, float ]
         H-band magnitude and uncertainty [mag]. Default None. 
        [ kmag, kmag_err : float, float ]
         K-band magnitude and uncertainty [mag]. Default None.
        HISTORY:                                                                                            
        Created 8 sep 20
        Joel Zinn (j.zinn@unsw.edu.au)
        '''
        self.plx = plx
        self.plx_err = plx
        self.logg_spec = logg_spec
        self.logg_spec_err = logg_spec_err
        self.teff_spec = teff_spec
        self.teff_spec_err = teff_spec_err
        
        self.jmag = jmag
        self.jmag_err = jmag_err
        self.hmag = hmag
        self.hmag_err = hmag_err
        self.kmag = kmag
        self.kmag_err = kmag_err
        
        # from Pinsonneault et al. 2018
        self.teff_sun = 5772. 
        self.dnu_sun = 135.146                                                                               
        self.numax_sun = 3076.                                                                               
        self.logg_sun = 2.7413e4   
        
    def gaiascalnmx(self, mass=1., AK=None, N_samples=1000):                                     
        """                                                                                                 
        Evaluate a prior on numax based on 2MASS magnitudes and Gaia parallax                               
        INPUTS:                                                                                              
        [ plx, plx_err, jmag, jmag_err, hmag, hmag_err, kmag, kmag_err ] : [ float, float, float, float, float, float, float, float ]
         These need to be defined in __init__().
        [ mass : float ]
         Optional mass prior option (not yet implemented!!!). Default 1.               
        [ AK : float ]
         Optional K band extinction. Default None.                                                               
        [ N_samples : int ]
         Number of samples from the prior to take and then return. Default 1000.        
        OUTPUTS:                                                                                             
        (numax_median, numax_std), numax_samp : (float, float), float ndarray
         Numax summary stats. and sample distribution [uHz].
        HISTORY:                                                                                            
        Written - Mackereth - 08/09/2020 (UoB @ online.tess.science)
        Modified JCZ 8 sep 20
        """                                                                                                 
        means = np.array([self.jmag, self.hmag, self.kmag, self.plx])                                                   
        cov = np.zeros((4,4))                                                                               
        cov[0,0] = self.jmag_err**2                                                                                  
        cov[1,1] = self.hmag_err**2                                                                                  
        cov[2,2] = self.kmag_err**2                                                                                  
        cov[3,3] = self.plx_err**2                                                                           
        multi_norm = multivariate_normal(means, cov)                                                        
        samples = multi_norm.rvs(size=N_samples)                                                            
        Jsamp, Hsamp, Ksamp, parallaxsamp = samples[:,0], samples[:,1], samples[:,2], samples[:,3]          
        numaxsamp = prob.numax_from_JHK(Jsamp, Hsamp, Ksamp, parallaxsamp, mass=mass, AK=AK)                
        numax_median = np.nanmedian(numaxsamp)                                                                     
        numax_std = np.nanstd(numaxsamp)                                                                     
        return (numax_median, numax_std), numaxsamp   
    

    #@staticmethod
    def numax(self, logg, teff):
        '''
        Return an expected numax given a log g and teff                                                   
        INPUTS:                                                                                           
        self.logg, self.logg_spec : float, float
         log10 surface gravity and uncertainty [cgs].                                                                     
        self.teff_spec, self.teff_spec_err : float, float                                                                                        
         effective temperature and uncertainty [K].                                                                         
        [ emp : bool ]                                                                                      
        OUTPUTS:                                                                                             
        numax : float                                                                                       
         Frequency of maximum oscillation [muhz].
        '''
        
        numax = 10.**(logg)/(self.logg_sun)*self.numax_sun*(teff/self.teff_sun)**(-0.5) 
        return numax
    
    def specnmx(self, N_samples=1000):                                                         
        '''                                                                                                 
        Return an expected numax, uncertainty, and numax samples, given a log g and teff                                                   
        INPUTS:                                                                                           
        self.logg, self.logg_spec : float, float
         log10 surface gravity and uncertainty [cgs].                                                                     
        self.teff_spec, self.teff_spec_err : float, float                                                                                        
         effective temperature and uncertainty [K].                                                                                    
        [ N_samples : int ]
         Number of samples to draw for numax samples. Default 1000.
        OUTPUTS:                                                                                             
        (numax_median, numax_std), numax_samp : (float, float), float ndarray
         Numax summary stats. and sample distribution [uHz].
         '''  
        #assert (not self.logg_spec)
        #assert (not None self.logg_spec_err)
        #assert is not None self.teff_spec
        #assert is not NOne self.teff_spec_err
        assert self.logg_spec > -99
        assert self.logg_spec_err > 0
        assert self.teff_spec > 0
        assert self.teff_spec_err > 0
        
        means = np.array([self.logg_spec, self.teff_spec])     
        cov = np.zeros((2,2))                                                                               
        cov[0,0] = self.logg_spec_err**2                                                                                  
        cov[1,1] = self.teff_spec_err**2                                                                                                                                                           
        multi_norm = multivariate_normal(means, cov)                                                        
        samples = multi_norm.rvs(size=N_samples)                                                            
        logg_samp, teff_samp = samples[:,0], samples[:,1]          
        numaxsamp = self.numax(logg_samp, teff_samp)          
        numax_median = np.median(numaxsamp)                                                                     
        numax_sigma = np.std(numaxsamp)                                                                     
        return (numax_median, numax_sigma), numaxsamp   

# these are real spec. and phot. data from an anonymous TESS star with measured numax of ~30uHz, with made-up uncertainties.
def get_gaiascalnmx():
    nup = NuPrior(plx=0.44, plx_err=0.01, jmag=10.64, jmag_err=0.01, hmag=10.134, hmag_err=0.01, kmag=10.02, kmag_err=0.01)
    print('(numax_median, numax_std, numax_samples) from gaiascalnmx:')
    print(nup.gaiascalnmx(mass='giants'))
    
def get_specnmx():
    nup = NuPrior(teff_spec=4900., teff_spec_err=100., logg_spec=2.4, logg_spec_err=0.1)
    print('(numax_median, numax_std, numax_samples) from gaiascalnmx:')
    print(nup.specnmx())
    
#get_gaiascalnmx() 
#get_specnmx()

    

In [19]:

tic261154649=NuPrior(teff_spec=4947., teff_spec_err=92., logg_spec=2.4565, logg_spec_err=0.05, plx=1.44, plx_err=0.0, 
    jmag=8.293, jmag_err=0.03, hmag=7.694, hmag_err=0.023, kmag=7.559, kmag_err=0.027)


In [50]:
a=tic261154649.gaiascalnmx(mass='giants')
print( a[0][1])

81.5702274552489


In [18]:
print(tic261154649.specnmx())

((34.70656713038831, 3.8969405216344564), array([30.46906406, 32.86286913, 41.93425964, 34.29828021, 34.2766918 ,
       36.25089793, 39.43914437, 32.10538126, 28.89233417, 32.40024675,
       46.26922138, 36.29794311, 31.11921528, 39.07764555, 33.33028027,
       42.091141  , 33.76119055, 33.08053416, 38.92242035, 39.56569299,
       42.69382701, 36.28568768, 34.75696225, 34.96347706, 35.94634659,
       33.26834576, 32.63078382, 31.28491326, 42.10268725, 37.41092367,
       33.87423664, 32.20625838, 39.47672369, 35.66520329, 36.22720569,
       38.25074383, 31.05819007, 30.56981497, 35.83402922, 33.74299002,
       36.81775354, 33.75130481, 39.9412526 , 31.54085455, 34.940489  ,
       36.21419841, 25.82647584, 34.12284737, 31.67487259, 29.50253733,
       34.77483049, 32.40837156, 33.94640595, 39.0519613 , 40.80343243,
       30.18156563, 40.54546099, 41.25169162, 35.96690396, 37.33281319,
       28.99061416, 38.64094043, 47.99539166, 39.01616707, 33.91409856,
       30.40744944, 34

In [21]:
infile='DR16_APOTIC_NS_TMshort2.txt'

In [22]:
outfile = infile.replace('.txt','_out.txt')

In [39]:
stars=np.loadtxt(infile, usecols=(0,3,4,5,6,7,8,9,10,11,12,13,14), skiprows=1)

In [40]:
ids,ateff,atefferr,alogg,aloggerr, gparallax, gparallaxerr, tj, tjerr,th, therr,tk, tkerr=zip(*stars) 

In [58]:
open(outfile,'w').write('## TICID  specnumax  specnumaxerr gaianumax  gaianumaxerr \n')

59

In [64]:
for i in range(len(ids)):
    star=NuPrior(teff_spec=ateff[i], teff_spec_err=atefferr[i], logg_spec=alogg[i], logg_spec_err=aloggerr[i], plx=gparallax[i], plx_err=gparallaxerr[i], 
        jmag=tj[i], jmag_err=tjerr[i], hmag=th[i], hmag_err=therr[i], kmag=tk[i], kmag_err=tkerr[i])
 
    if gparallax[i] > 0:
        gaiastar=star.gaiascalnmx(mass='giants')
        gaianumaxstar=gaiastar[0][0]
        gaianumaxrangestar=gaiastar[0][1]
    else:
        gaianumaxstar=-9999
        gaianumaxrangestar=-9999

    if (ateff[i] > 0 and alogg[i] > -10):
        specstar=star.specnmx()
        specnumaxstar=specstar[0][0]
        specnumaxrangestar=specstar[0][1]
    else:
        specnumaxstar=-9999
        specnumaxrangestar=-9999   
        
    lin='   %.0f'%ids[i]+'\t'+'   %.6f'%specnumaxstar+'\t'+'   %.6f'%specnumaxrangestar+'\t'+'   %.6f'%gaianumaxstar+'\t'+'   %.6f'%gaianumaxrangestar+'\n'
    open(outfile,'a').write(lin)     